In [16]:
import re
import nltk
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

plt.style.use('default')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SamuelPorto\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

O database escolhido é está disponível no kaggle como: Coronavirus tweets NLP - Text Classification (https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification).

Esse database contém tweets postados na pandemia, junto de data, localização e classificação do post. Sendo a classificação uma variável categórica ordinal (Extremely Negative, Negative, Neutral, Positive e Extremely Positive) onde os tweets negativos passam informações de insegurança e medo causado por conta do vírus e os positivos são o oposto.

Por motivos de agilidade de implementação, testes e análises neste documentos vamos considerar apenas duas categorias Negativo que encapsula as originais categorias Extremely Negative e Negative; e Positivo correspondendo as categorias Positive, Extremely Positive e Neutral.

In [3]:
train = pd.read_csv('Corona_NLP_train.csv', encoding = 'latin1')
test = pd.read_csv('Corona_NLP_test.csv', encoding = 'latin1')

In [4]:
def categorias(x):
    if x ==  "Extremely Positive":
        return "1"
    elif x == "Extremely Negative":
        return "1"
    elif x == "Negative":
        return "0"
    elif x ==  "Positive":
        return "0"
    else:
        return "1"

train['label']=train['Sentiment'].apply(lambda x:categorias(x))
test['label']=test['Sentiment'].apply(lambda x:categorias(x))

In [5]:
train.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,label
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,1
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive,0
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive,0
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive,0
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative,1


In [6]:
train.label.value_counts(normalize=True)

0    0.518478
1    0.481522
Name: label, dtype: float64

# Data Prep

Vamos limpar a fontes de dados para podermos ter um melhor desempenho na modelagem.

In [7]:
# Faz o filtro de stop words, pontuações, POS e padroniza o texto
def filter_serie(serie):

    # Padrões regex que serão filtrados nos comentarios
    hyperlink = '(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-&?=%.]+'
    date = '([0-2][0-9]|(3)[0-1])(\/|\.)(((0)[0-9])|((1)[0-2]))(\/|\.)\d{2,4}'
    dinheiro = '[R]{0,1}\$[ ]{0,}\d+(,|\.)\d+'
    negacao = '([nN][ãÃaA][oO]|[ñÑ]| [nN] )'
    mencao = '@\w+'
    hash = '#\w+'
    emoji = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)

    sw = stopwords.words('portuguese')
    sw.append('\n')
    sw.append('\r')

    # Aplica o regex para remover links
    serie = serie.apply(lambda x: ' '.join(
        [re.sub(hyperlink, ' link ', word) for word in x.split()]))

    # Deixa todos os reviews em lower case
    serie = serie.apply(lambda x: ' '.join(
        [word for word in x.lower().split() if word not in (sw)]))

    # Aplica o regex para remover mencoes
    serie = serie.apply(lambda x: ' '.join(
        [re.sub(mencao, '', word) for word in x.split()]))

    # Aplica o regex para remover hashtags
    serie = serie.apply(lambda x: ' '.join(
        [re.sub(hash, '', word) for word in x.split()]))

    # Aplica o regex para remover emojis
    serie = serie.apply(lambda x: ' '.join(
        [re.sub(emoji, '', word) for word in x.split()]))

    # Aplica o regex para remover datas
    serie = serie.apply(lambda x: ' '.join(
        [re.sub(date, ' date ', word) for word in x.split()]))

    # Aplica o regex para remover valores monetários
    serie = serie.apply(lambda x: ' '.join(
        [re.sub(dinheiro, ' money ', word) for word in x.split()]))

    # Aplica o regex para padronizar negações
    serie = serie.apply(lambda x: ' '.join(
        [re.sub(negacao, ' not ', word) for word in x.split()]))

    # Aplica regex para remover caracteres especiais
    serie = serie.apply(lambda x: ' '.join(
        [re.sub('\W', ' ', word) for word in x.split()]))

    # Remove as pontuações
    serie = serie.apply(lambda x: ''.join(
        [word for word in x if word not in (string.punctuation)]))

    # Aplica regex para remover espaçamento a mais no início de cada review
    serie = serie.apply(lambda x: ' '.join(
        [re.sub('\s+', ' ', word) for word in x.split()]))

    # Aplica regex para remover espaçamento a mais no final de cada review
    serie = serie.apply(lambda x: ' '.join(
        [re.sub('[ \t]+$', '', word) for word in x.split()]))

    return serie

In [8]:
train['text'] = filter_serie(train['OriginalTweet'])
test['text'] = filter_serie(test['OriginalTweet'])

In [9]:
train[['OriginalTweet','text', 'label']].head()

,OriginalTweet,text,label
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,link and link and link,1
1,advice Talk to your neighbours family to excha...,advice talk to your neighbours family to excha...,0
2,Coronavirus Australia: Woolworths to give elde...,coronavirus australia woolworths to give elder...,0
3,My food stock is not the only one which is emp...,my food stock is not the only one which is lin...,0
4,"Me, ready to go at supermarket during the #COV...",me ready to go at supermarket during the outbr...,1


# Naive Bayes

In [17]:
vectorizer = CountVectorizer(binary=True)

X_train = vectorizer.fit_transform(train['text'])
X_test = vectorizer.transform(test['text'])

y_train = train['label']
y_test = test['label']

In [18]:
model = BernoulliNB()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.59      0.59      0.59      1988
           1       0.55      0.54      0.54      1810

    accuracy                           0.57      3798
   macro avg       0.57      0.57      0.57      3798
weighted avg       0.57      0.57      0.57      3798



aulas:
11
12
17
18